In [1]:
# %matplotlib inline
%matplotlib widget

In [2]:
%load_ext autoreload
%autoreload 2

# IMPORT

In [3]:
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import xarray as xr


from pycns import CnsStream, CnsReader, get_viewer
import physio

from physio_rea_analyse import *
from custom_view import *

from configuration import *
from tools import *

# READER

In [4]:
# patient_id = 'P50'
# patient_id = 'MF12'
# patient_id = 'BM3' # periodic oscillations RSA 2022-09-11T07:50 + trigging, EEG pas ok tout le long car parfois devient 6Hz + 50Hz only , 6 Hz at 2022-09-10T03:16:03.013765
patient_id = 'GA9' # periodic oscillations RSA 2021-08-27T03:00:00, EEG OK tout le long // 2021-08-29T14:26:34 = muscle artifact EEG
# patient_id = 'HA1' # ses CSD sont cooccurantes avec des evenements de PA et de PIC et la RSA aussi. Pointes EEG chelou
# patient_id = 'FC13' # EEG mauvais parfois et devient 6 Hz du coup
# patient_id = 'MJ18' # No CO2
# patient_id = 'WJ14' # periodic oscillations RSA trigging
# patient_id = 'P1' 
# patient_id = 'P3' 
# patient_id = 'P11' 
# patient_id = 'P14' # gros contenu fréquentiel respi dans coeur abp et icp
# patient_id = 'P27' 
# patient_id = 'JL5'
# patient_id = 'PL20'
# patient_id = 'SP2'
raw_folder = data_path / patient_id
cns_reader = CnsReader(raw_folder, with_events=True, event_time_zone='Europe/Paris')
cns_reader.streams

{'PLETH': CnsStream PLETH  rate:122Hz  shape:(59387608,),
 'SpO2': CnsStream SpO2  rate:1Hz  shape:(456153,),
 'EEG_Impedance': CnsStream EEG_Impedance  rate:1Hz  shape:(479832, 22),
 'ABP_Mean': CnsStream ABP_Mean  rate:1Hz  shape:(460234,),
 'ICP': CnsStream ICP  rate:123Hz  shape:(59576040,),
 'CPP': CnsStream CPP  rate:1Hz  shape:(459867,),
 'ECG_II': CnsStream ECG_II  rate:492Hz  shape:(238307616,),
 'ICP_Mean': CnsStream ICP_Mean  rate:1Hz  shape:(461586,),
 'EtCO2': CnsStream EtCO2  rate:1Hz  shape:(396079,),
 'EEG': CnsStream EEG  rate:256Hz  shape:(122836639, 19),
 'CO2': CnsStream CO2  rate:122Hz  shape:(59642192,),
 'ABP_Dias': CnsStream ABP_Dias  rate:1Hz  shape:(453950,),
 'ABP': CnsStream ABP  rate:123Hz  shape:(59507428,),
 'EEG_NeonatalParams': CnsStream EEG_NeonatalParams  rate:2Hz  shape:(959723, 16),
 'HR': CnsStream HR  rate:1Hz  shape:(457716,),
 'ABP_Syst': CnsStream ABP_Syst  rate:1Hz  shape:(453950,)}

In [5]:
meta = pd.read_excel(base_folder / 'overview_data_pycns.xlsx').set_index('Patient').loc[patient_id,:][18:]
meta

N_SDs            2
motif           TC
GCS_sortie    15.0
mRS_sortie     5.0
mRS_1mois      4.0
mRS_3mois      3.0
mRS_6mois      3.0
Name: GA9, dtype: object

In [6]:
pd.DataFrame(cns_reader.events)

,name,start_time,duration,description
0,1:debut eeg,2021-08-25 22:51:23.570,0.000,
1,mu + rythme de volet,2021-08-25 22:55:58.528,0.000,
2,mu?,2021-08-25 23:03:18.746,0.000,
3,pointe lente C4,2021-08-25 23:26:40.298,0.000,
4,PL C4,2021-08-25 23:28:24.191,0.000,
5,pointe C4,2021-08-25 23:32:44.642,0.000,
6,pointe C4,2021-08-26 00:18:26.692,0.000,
7,debut TDM,2021-08-26 09:25:19.481,0.000,Pd de reconnection des electrodes d'ECoG
8,retour TDM,2021-08-26 10:25:51.481,0.000,
9,pointe C4 répétées / mu ?,2021-08-26 14:37:36.986,0.000,


# PRECOMPUTE

In [24]:
cns_reader

CnsReader: MF12
17 streams : ['PLETH', 'SpO2', 'EEG_Impedance', 'ABP_Mean', 'Temp', 'ICP', 'CPP', 'ECG_II', 'ICP_Mean', 'EEG', 'EtCO2_Ext', 'CO2', 'ABP_Dias', 'ABP', 'EEG_NeonatalParams', 'HR', 'ABP_Syst']
Events: 51

In [25]:
force_compute = False
ecg_stream = cns_reader.streams['ECG_II']
if force_compute:
    dates = ecg_stream.get_times()
    ecg, r_peaks = detect_ecg_peak(ecg_stream)
    r_peaks['peak_date'] = dates[r_peaks['peak_index']]
else:
    r_peaks = detect_ecg_job.get(patient_id).to_dataframe()
print(r_peaks.shape)

(514427, 3)


In [26]:
force_compute = False

co2_stream = cns_reader.streams['CO2']
if force_compute:
    dates = co2_stream.get_times()
    resp_cycles = detect_resp_cycles(co2_stream)
    resp_cycles['inspi_date'] = dates[resp_cycles['inspi_index']]
    resp_cycles['expi_date'] = dates[resp_cycles['expi_index']]
else:
    resp_cycles = detect_resp_job.get(patient_id).to_dataframe()
print(resp_cycles.shape)

(119548, 13)


In [27]:
force_compute = False

if force_compute:
    rsa_cycles, cyclic_cardiac_rate = physio.compute_rsa(
        resp_cycles,
         r_peaks,
         srate=10.,
         two_segment=True,
         points_per_cycle=50,
    )
    del cyclic_cardiac_rate
    rsa_cycles['date'] = resp_cycles['inspi_date']
else:
    rsa_cycles = rsa_job.get(patient_id).to_dataframe()
print(rsa_cycles.shape)

(119548, 13)


In [28]:
force_compute = False

if force_compute:
    abp_stream = cns_reader.streams['ABP']
    srate_abp = abp_stream.sample_rate
    raw_abp, dates = abp_stream.get_data(with_times=True, apply_gain=True)
    abp_features = compute_abp(raw_abp, srate_abp, date_vector = dates)
else:
    abp_features = detect_abp_job.get(patient_id).to_dataframe()
print(abp_features.shape)

(534764, 16)


In [29]:
force_compute = False

if force_compute:
    icp_stream = cns_reader.streams['ICP']
    srate_icp = icp_stream.sample_rate
    raw_icp, dates = icp_stream.get_data(with_times=True, apply_gain=True)
    icp_features = compute_icp(raw_icp, srate_icp, date_vector = dates)
else:
    icp_features = detect_icp_job.get(patient_id).to_dataframe()
print(icp_features.shape)

(204523, 16)


In [30]:
force_compute = False

if force_compute:
    resp_cycles = detect_resp_job.get(patient_id).to_dataframe()
    inspi_ratio = resp_cycles['cycle_ratio'].median()
    cycle_times = resp_cycles[['inspi_time', 'expi_time', 'next_inspi_time']].values

    ecg_peaks_resp = detect_ecg_job.get(patient_id).to_dataframe()

    rpeak_phase_resp = physio.time_to_cycle(ecg_peaks_resp['peak_time'].values, cycle_times, segment_ratios=[inspi_ratio])
    ecg_peaks_resp['rpeak_resp_phase'] = rpeak_phase_resp
    ecg_peaks_resp['resp_phase'] = rpeak_phase_resp % 1
    ecg_peaks_resp['resp_cycle'] = np.floor(rpeak_phase_resp)
else:
    ecg_peaks_resp = crps_job.get(patient_id).to_dataframe()
print(ecg_peaks_resp.shape)

(514427, 6)


In [52]:
# stream_names = ['ABP','CO2']
# srate_traube = 2
# ds = cns_reader.export_to_xarray(stream_names, resample=True, sample_rate=srate_traube)

# VIEWER RAW

In [10]:
cns_reader.streams.keys()

dict_keys(['ART_Dias', 'rSO2_Right', 'rSO2_Left', 'SpO2', 'ART', 'EEG_Impedance', 'ICP', 'CPP', 'ECG_II', 'ICP_Mean', 'ART_Mean', 'EtCO2', 'EEG', 'CO2', 'PbtO2', 'EEG_NeonatalParams', 'HR', 'ART_Syst'])

In [14]:
w = get_viewer(cns_reader, stream_names=['CO2'], with_events = False)
w

Viewer(children=(VBox(children=(HBox(children=(Button(description='autoscale', icon='refresh', style=ButtonSty…

# VIEWER PHYSIO

In [59]:
patient_id

'MF12'

In [60]:
%matplotlib widget

rate_bins_resp = np.arange(5, 30, 0.5)
step_bins_ecg = 1
eeg_chan = 'F3'

ext_plots = {
            # ABP_Detections(cns_reader.streams['ABP'], abp_features),
            # ICP_Detections(cns_reader.streams['ICP'], icp_features),
            
            # Wavelet_Power(cns_reader.streams['EEG'], eeg_chan, f_start = 4, f_stop = 8, n_steps = 30, n_cycles = 15, scaling = None),
            # Bipolar(eeg_stream, 'ECoGA4', 'ECoGA3', lowcut = 0.0008, highcut = 0.5, down_sampling_factor = 100),
            # AC_Envelope(eeg_stream, eeg_chan, highcut_amp=0.01),
            # Spreading_depol_bipol(cns_reader.streams['EEG'], down_sampling_factor = 10),
            # Depol(eeg_stream, eeg_chan, min_amp_mv = 15, max_amp_mv = None, min_duration_sec = 60, reverse_sig = False),
            # Spectrogram_bio(co2_stream, wsize = 300, lf = 0.1, hf = 0.4, overlap_prct = 0.75),
            # Spectrogram_eeg(cns_reader.streams['EEG'], eeg_chan, wsize = 10, lf = 1, hf = 20),
            'resp':Respi_Rate(resp_cycles, resp_wsize_in_mins = 3, ratio_sat = 4, rate_bins_resp=rate_bins_resp),
            'heart':Heart_Rate(r_peaks, step_bins_ecg = step_bins_ecg, hrv_wsize_in_mins = 3, ratio_sat = 3, plot_type = '2d'),
            'RSA':RSA(rsa_cycles, win_cycles_smooth = 10, n_mads_cleaning = 5), 
            # Pulse_Pressure(abp_features, n_mads_cleaning = 2),
            # Traube_Herring(ds['ABP'].values, ds['CO2'].values, srate_traube, ds['times'].values, wsize = 60), 
            'CRPS_2d':CRPS(ecg_peaks_resp, plot_type = '2d'),
            'CRPS_1d':CRPS(ecg_peaks_resp, plot_type = '1d', step_secs_1d=60),
            'HR_Ratio':HeartResp_FreqRatio(ecg_peaks_resp, resp_cycles, step_secs=60),
}

# w = get_viewer(cns_reader, stream_names=['ECG_II','CO2'], ext_plots=ext_plots)
w = get_viewer(cns_reader, stream_names=['CO2'], ext_plots=ext_plots)
w


Viewer(children=(VBox(children=(HBox(children=(Button(description='autoscale', icon='refresh', style=ButtonSty…

/crnldata/tiger/baptiste.balanca/Neuro_rea_monitorage/scripts_analyse_physio_neuro_rea/tools.py:18: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.where(a != 0, a * np.log(a / b), 0))
/crnldata/tiger/baptiste.balanca/Neuro_rea_monitorage/scripts_analyse_physio_neuro_rea/tools.py:18: RuntimeWarning: invalid value encountered in multiply
  return np.sum(np.where(a != 0, a * np.log(a / b), 0))
/home/valentin.ghibaudo/.virtualenvs/py3.11_cluster/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/valentin.ghibaudo/.virtualenvs/py3.11_cluster/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/crnldata/tiger/baptiste.balanca/Neuro_rea_monitorage/scripts_analyse_physio_neuro_rea/custom_view.py:605: RuntimeWarning: invalid value encountered in divide
  count = count / n

# VIEWER MORLET NEURO

In [9]:
%matplotlib widget

scalp_chan = 'F3'
ecog_chan = 'ECoGA3'

ext_plots = {
            'morlet':Wavelet_Power(cns_reader.streams['EEG'], ecog_chan,  f_start = 4, f_stop = 20, n_steps = 40, n_cycles = 5, amplitude_exponent=1, scaling = None),
}

w = get_viewer(cns_reader, stream_names=['ICP','ABP','EEG'], ext_plots=ext_plots)
w


Viewer(children=(VBox(children=(HBox(children=(Button(description='autoscale', icon='refresh', style=ButtonSty…

# VIEWER EEG ARTIFACT

In [ ]:
precomputed_muscle_artifacts = local_muscle_artifacts_job.get(patient_id)['local_muscle_artifacts']
# artifacts_vector = precomputed_muscle_artifacts.loc['Scalp',:].values
artifacts_vector = precomputed_muscle_artifacts.loc['ECoG',:].values

In [ ]:
ylim = (-1, 10)
threshold = 5

In [ ]:
dates_artifacts = precomputed_muscle_artifacts['date'].values

In [ ]:
eeg_chans = [chan for chan in cns_reader.streams['EEG'].channel_names if not 'ECoG' in chan]
ecog_chans = [chan for chan in cns_reader.streams['EEG'].channel_names if 'ECoG' in chan]

In [14]:
%matplotlib widget

scalp_chan = 'F3'
ecog_chan = 'ECoGA5'

ext_plots = [
            # Spectrogram_eeg(cns_reader.streams['EEG'], scalp_chan, wsize = 30, lf = 2, hf = 12, scaling = None, saturation_quantile = 0.02, cmap = 'viridis'),
            # Monopolar(cns_reader.streams['EEG'], scalp_chan, down_sampling_factor = 2, lowcut = 0.01, highcut = 50),
            # Monopolar(cns_reader.streams['EEG'], scalp_chan, down_sampling_factor = 2),
            # Bipolar(cns_reader.streams['EEG'], 'Fz', 'Cz', down_sampling_factor = 2, lowcut = 0.01, highcut = 50),
            # Bipolar(cns_reader.streams['EEG'], 'Cz', 'Pz', down_sampling_factor = 2, lowcut = 0.1, highcut = 30),
            # Bipolar(cns_reader.streams['EEG'], 'ECoG4', 'ECoG3', down_sampling_factor = 2, lowcut = 0.01, highcut = 50),
            Wavelet_Power(cns_reader.streams['EEG'], ecog_chan,  f_start = 2, f_stop = 12, n_steps = 40, n_cycles = 5, amplitude_exponent=1, scaling = None),
            # Total_power(cns_reader.streams['EEG'], ecog_chan,  wsize = 30),
            # Spectrum_Slope(cns_reader.streams['EEG'], ecog_chan,  wsize = 30),
            # Total_power(cns_reader.streams['EEG'], ecog_chan,  wsize = 5),
            # Spectrum_Slope(cns_reader.streams['EEG'], ecog_chan,  wsize = 5),
            # AC_Envelope(cns_reader.streams['EEG'], scalp_chan, lowcut_ac = 30, highcut_ac = 127, highcut_amp = 1, down_sampling_factor = 2),
            # Muscle_artifacts(cns_reader.streams['EEG'], used_chans = eeg_chans,  lowcut = 30, highcut_zscore = 1),
            # Muscle_artifacts_precompute(artifacts_vector, dates_artifacts, threshold, ylim, mode = 'bool'),
            # Artifact_6Hz(cns_reader.streams['EEG'], scalp_chan ),
            ]

w = get_viewer(cns_reader, stream_names=['ABP','ICP','CO2','EEG'], ext_plots=ext_plots)
w


Viewer(children=(VBox(children=(HBox(children=(Button(description='autoscale', icon='refresh', style=ButtonSty…

In [ ]:
ev_ind = 31

win_jitter = np.timedelta64(10, 'm')
time_ev = np.datetime64(evs.loc[ev_ind,'StartTimeGMT'])
new_time = time_ev - win_jitter

print(evs.loc[ev_ind,'Name'])
print(time_ev)
print(evs.loc[ev_ind,'Duration'])

w.time_slider.update_time(new_time=new_time)


In [ ]:
evs

In [ ]:
# MF12 T7 le 2022-09-13T07:54:08.815104, signal OK
# MF12 T7 le 2022-09-13T08:10:08.815104, signal artefacté car artefact à 08h07 et 53 secs

# VIEWER CSD

In [19]:
%matplotlib widget

bipol_ecog = 'ECoGA3-ECoGA2'
mono_ecog = 'ECoGA3'

# bipol_ecog = 'ECoG3-ECoG2'
# mono_ecog = 'ECoG3'

ext_plots = {
            'CSDs_mono':Spreading_depol_mono(cns_reader.streams['EEG'], detections = pd.DataFrame(cns_reader.events), lowcut_dc = 0.002, highcut_dc = 0.1, global_gain = 0.05),
            # 'CSDs_dipol':Spreading_depol_bipol(cns_reader.streams['EEG'], detections = pd.DataFrame(cns_reader.events), lowcut_dc = 0.002, highcut_dc = 0.1),
            # 'CSDs_mono':Spreading_depol_mono(cns_reader.streams['EEG'], detections = depol_detections, lowcut_dc = 0.002, highcut_dc = 0.1),
            # 'Morlet':Wavelet_Power(stream=cns_reader.streams['EEG'], chan='ECoGA3',  f_start = 1, f_stop=100, n_steps=50, n_cycles=20),
            # 'CSDs_bipol':Spreading_depol_bipol2(cns_reader.streams['EEG'], detections = pd.DataFrame(cns_reader.events), down_sampling_factor = 10, lowcut_dc = 0.005),
            # 'Spectro':Spectrogram_eeg(stream = cns_reader.streams['EEG'], chan_name='ECoG4', wsize=20, lf=2, hf=100, scaling = 'log', saturation_quantile = 0.01),
            # Spreading_depol_scalp(cns_reader.streams['EEG'], down_sampling_factor = 10),
            # 'AC_amp1':Amplitude_Envelope(cns_reader.streams['EEG'], 'ECoGA3' ,lowcut = 0.5, highcut = 40, highcut_amp = 0.01),
            # 'AC_amp2':Amplitude_Envelope(cns_reader.streams['EEG'], 'ECoGA4' ,lowcut = 0.5, highcut = 40, highcut_amp = 0.5),
            # 'AC_amp2':Amplitude_Envelope(cns_reader.streams['EEG'], 'ECoGA4' ,lowcut = 25, highcut = 80, highcut_amp = 0.02),
            # 'AC_amp_bipol':Amplitude_Envelope(cns_reader.streams['EEG'], 'ECoGA3-ECoGA2' ,lowcut = 0.5, highcut = 40, highcut_amp = 0.02),
            # SD_Detection(cns_reader.streams['EEG'], mono_ecog, down_sampling_factor = 100, lowcut = 0.001, highcut = 0.005, threshold_dc = 10, threshold_ac = 0.3)
            # Respi_Rate(resp_cycles, resp_wsize_in_mins = 3, ratio_sat = 4, rate_bins_resp=np.arange(5, 30, 0.5)),
            # Heart_Rate(r_peaks, step_bins_ecg = 1, hrv_wsize_in_mins = 3, ratio_sat = 3, plot_type = '2d'),
            # RSA(rsa_cycles, win_cycles_smooth = 10, n_mads_cleaning = 5),
            # 'total_power':Total_power(cns_reader.streams['EEG'], 'ECoGA3',  wsize = 30),
            # 'Slope':Spectrum_Slope(cns_reader.streams['EEG'], 'ECoGA4',  wsize = 5),
            'Envelope_2D_SD':Amplitude_Envelope_MultiChan(cns_reader.streams['EEG'], chan_type = 'ECoG' , lowcut =  0.001, highcut = 0.01, quantile_saturation = 0.01, exclude_chans = None),
            'Envelope_1D_SD':Amplitude_Envelope_MultiChan(cns_reader.streams['EEG'], chan_type = 'ECoG' , lowcut =  0.001, highcut = 0.01, quantile_saturation = 0.01, plot_mode = '1D', exclude_chans = None),
            # 'DC_detection3':Amplitude_Envelope_Show_Automated_Detection(cns_reader.streams['EEG'], 'ECoGA3', depolarization_detection_job.get(patient_id, 'ECoGA3').to_dataframe()),
            # 'DC_detection4':Amplitude_Envelope_Show_Automated_Detection(cns_reader.streams['EEG'], 'ECoGA4', depolarization_detection_job.get(patient_id, 'ECoGA4').to_dataframe()),
}

w = get_viewer(cns_reader, stream_names=['ECG_II'], ext_plots=ext_plots, with_events=True)
w

Viewer(children=(VBox(children=(HBox(children=(Button(description='autoscale', icon='refresh', style=ButtonSty…

# VIEWER PRX

In [ ]:
prx_processed_streams = CnsReader(data_path / patient_id / 'Processed')

In [ ]:
stream_list = list(prx_processed_streams.streams.keys())

In [ ]:
prx_homemade = compute_homemade_prx(cns_reader, win_size_rolling_mins=1)

In [ ]:
print(stream_list)
print(len(stream_list))

In [39]:
%matplotlib widget


ext_plots = {
            'ppc':Cereral_Perfusion_Pressure(cns_reader.streams['ICP_Mean'],cns_reader.streams['ABP_Mean']),
            'heart_resp_icp':Heart_Resp_in_ICP(cns_reader.streams['ICP'], wsize_secs=30, resp_band = (0.1, 0.5), heart_band = (0.8, 2.5)),
            'heart_resp_abp':Heart_Resp_in_ABP(cns_reader.streams['ABP'], wsize_secs=30, resp_band = (0.1, 0.5), heart_band = (0.8, 2.5)),
            # 'heart_resp_freq':Heart_Resp_freq_from_ICP(cns_reader.streams['ICP'], wsize_secs=30, resp_band = (0.1, 0.5), heart_band = (0.8, 2.5)),
            # 'prx1':PRx(prx_r, dates_prx),
            # 'prx2':PRx(prx_r_nans, dates_prx_nans),
            # 'prx_processed':PRX_Processed(prx_processed_streams.streams[stream_list[4]]),
            # 'prx_homemade':PRx_homemade(prx_df=prx_homemade),
            # 'spectro_icp':Spectrogram_bio(cns_reader.streams['ICP'], wsize = 30, lf = 0.1, hf = 3, power_or_amplitude = 'amplitude', saturation_quantile = 0.01, log_transfo = True),
            # 'spectro_abp':Spectrogram_bio(cns_reader.streams['ABP'], wsize = 30, lf = 0.1, hf = 3, power_or_amplitude = 'amplitude', saturation_quantile = 0.01, log_transfo = True),
            # 'wavelet_icp':Wavelet_Power_bio(cns_reader.streams['ABP'], f_start=0.15, f_stop=2, n_steps=50, n_cycles=10, amplitude_exponent=1, quantile_saturation = 0.01, scaling = None, compute_margin_secs = 30, down_samp_plot = 3),
}

w = get_viewer(cns_reader, stream_names=['ICP','ABP','ICP_Mean','ABP_Mean'], ext_plots=ext_plots)
w

Viewer(children=(VBox(children=(HBox(children=(Button(description='autoscale', icon='refresh', style=ButtonSty…

/crnldata/tiger/baptiste.balanca/Neuro_rea_monitorage/scripts_analyse_physio_neuro_rea/custom_view.py:1235: RuntimeWarning: invalid value encountered in divide
  ax2.plot(dates_spectrum, max_amplitudes[1] / max_amplitudes[0], color = 'k')
/crnldata/tiger/baptiste.balanca/Neuro_rea_monitorage/scripts_analyse_physio_neuro_rea/custom_view.py:1223: UserWarning: nperseg = 3748 is greater than input length  = 0, using nperseg = 0
  freqs, times_spectrum_s, Sxx_abp = scipy.signal.spectrogram(raw_abp, fs = srate, nperseg =  int(self.wsize_secs * srate), scaling = 'spectrum')


# VIEWER COMPLIANCE

In [41]:
%matplotlib widget


ext_plots = {
            'psi':PSI(psi_da=psi_job.get(patient_id)['psi']),
}

w = get_viewer(cns_reader, stream_names=['ICP','ICP_Mean'], ext_plots=ext_plots)
w

Viewer(children=(VBox(children=(HBox(children=(Button(description='autoscale', icon='refresh', style=ButtonSty…